# **ONNX Model Evaluation**

In [7]:
!pip install -q ultralytics

In [8]:
from ultralytics import YOLO

In [9]:
onnx_model = YOLO('best.onnx')

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


### **Inference on Test Images**

In [10]:
results = onnx_model('Split Dataset/test/images')

Loading best.onnx for ONNX Runtime inference...


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs



2024-06-02 03:50:12.070596226 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.11: cannot open shared object file: No such file or directory

2024-06-02 03:50:12.070635424 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:747 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.


image 1/1758 /mnt/batch/tasks/shared/LS_root/mounts/clusters/myinstance1/code/Split Dataset/test/images/1009.jpg: 640x640 1 Eastern Chipmunk, 51.8ms
image 2/1758 /mnt/batch/tasks/shared/LS_root/mounts/clusters/myinstance1/code/Split Dataset/test/images/1013.jpg: 640x640 1 Eastern Chipmunk, 46.6ms
image 3/1758 /mnt/batch/tasks/shared/LS_root/mounts/clusters/myinstance1/code/Split Dataset/test/images/1015.jpg: 640x640 1 Eastern Chipmunk, 37.6ms
image 4/1758 /mnt/batch/tasks/shared/LS_root/mounts/clusters/myinstance1/code/Split Dataset/test/images/1024.jpg: 640x640 2 Eastern Chipmunks, 36.3ms
image 5/1758 /mnt/batch/tasks/shared/LS_root/mounts/clusters/myinstance1/code/Split Dataset/test/images/1030.jpg: 640x640 1 Eastern Chipmunk, 37.2ms
image 6/1758 /mnt/batch/tasks/shared/LS_root/mounts/clusters/myinstance1/code/Split Dataset/test/images/1033.jpg: 640x640 1 Eastern Chipmunk, 44.1ms
image 7/1758 /mnt/batch/tasks/shared/LS_root/mounts/clusters/myinstance1/code/Split Dataset/test/images/1

## **IoU Scores for Individual Images**

In [56]:
import json
import numpy as np

def load_coco_annotations(json_file_path):
    with open(json_file_path, 'r') as f:
        coco_data = json.load(f)
    return coco_data

def get_actual_bbox_and_labels(coco_data, image_id):
    annotations = [anno for anno in coco_data['annotations'] if anno['image_id'] == image_id]
    bboxes = [anno['bbox'] for anno in annotations]
    labels = [anno['category_id'] for anno in annotations]
    return bboxes, labels

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area / union_area
    return iou

def convert_bbox_format(bbox):
    """Convert [x1, y1, x2, y2] to [x, y, width, height]."""
    x1, y1, x2, y2 = bbox
    return [x1, y1, x2 - x1, y2 - y1]

def compare_results(onnx_results, coco_annotations, image_id, iou_threshold=0.5):
    actual_bboxes, actual_labels = get_actual_bbox_and_labels(coco_annotations, image_id)
    
    results_object = onnx_results
    detected_boxes = results_object.boxes.xyxy.cpu().numpy()
    detected_class_indices = results_object.boxes.cls.cpu().numpy()
    detected_class_names = results_object.names
    
    iou_scores = []

    for i, detected_box in enumerate(detected_boxes):
        detected_class_index = int(detected_class_indices[i])
        detected_class_name = detected_class_names.get(detected_class_index, 'Unknown')
        detected_box_converted = convert_bbox_format(detected_box)
        
        for j, actual_bbox in enumerate(actual_bboxes):
            actual_label = actual_labels[j]
            iou = calculate_iou(detected_box_converted, actual_bbox)
            
            if iou >= iou_threshold:
                actual_class_name = detected_class_names.get(actual_label, 'Unknown')
                iou_scores.append((iou, detected_class_name == actual_class_name))
                print(f"Detected bbox: {detected_box}, Detected class: {detected_class_name}")
                print(f"Actual bbox: {actual_bbox}, Actual class: {actual_class_name}")
                print(f"IoU: {iou:.2f}")
                print(f"Match: {detected_class_name == actual_class_name}")
                print('-' * 50)
                
    return iou_scores

coco_json_file_path = 'test_annotations.json'
coco_annotations = load_coco_annotations(coco_json_file_path)

iou_scores_all_images = []

for idx, image_info in enumerate(coco_annotations['images']):
    image_id = image_info['id']
    print(f"Processing image ID: {image_id}")
    
    if idx < len(results):
        onnx_results = results[idx]  # Get the results for this specific image
        iou_scores = compare_results(onnx_results, coco_annotations, image_id)
        iou_scores_all_images.append((image_id, iou_scores))
    else:
        print(f"No ONNX results available for image ID: {image_id}")

# Print or process the IoU scores for all images
print("IoU scores for all images:")
for image_id, scores in iou_scores_all_images:
    print(f"Image ID: {image_id}, IoU scores: {scores}")

Processing image ID: 1009
Detected bbox: [     554.41      1123.7      686.16      1210.2], Detected class: Eastern Chipmunk
Actual bbox: [547.168256, 1104.893952, 146.8909568, 104.4705792], Actual class: Eastern Chipmunk
IoU: 0.73
Match: True
--------------------------------------------------
Processing image ID: 1013
Detected bbox: [     881.73      1261.6      1236.4      1426.7], Detected class: Eastern Chipmunk
Actual bbox: [870.311936, 1253.285376, 361.689088, 176.713728], Actual class: Eastern Chipmunk
IoU: 0.89
Match: True
--------------------------------------------------
Processing image ID: 1015
Detected bbox: [     1459.3        1250      1808.5      1405.1], Detected class: Eastern Chipmunk
Actual bbox: [1434.165248, 1240.422912, 454.797312, 169.411584], Actual class: Eastern Chipmunk
IoU: 0.70
Match: True
--------------------------------------------------
Processing image ID: 1024
Detected bbox: [     1712.1        1268      2046.9      1449.8], Detected class: Eastern Ch

## **Accuracy, Average IoU Score, F1 Score, Precision & Recall**

In [57]:
import json
import numpy as np

def load_coco_annotations(json_file_path):
    with open(json_file_path, 'r') as f:
        coco_data = json.load(f)
    return coco_data

def get_actual_bbox_and_labels(coco_data, image_id):
    annotations = [anno for anno in coco_data['annotations'] if anno['image_id'] == image_id]
    bboxes = [anno['bbox'] for anno in annotations]
    labels = [anno['category_id'] for anno in annotations]
    return bboxes, labels

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area / union_area
    return iou

def convert_bbox_format(bbox):
    """Convert [x1, y1, x2, y2] to [x, y, width, height]."""
    x1, y1, x2, y2 = bbox
    return [x1, y1, x2 - x1, y2 - y1]

def compare_results(onnx_results, coco_annotations, image_id, iou_threshold=0.5):
    actual_bboxes, actual_labels = get_actual_bbox_and_labels(coco_annotations, image_id)
    
    results_object = onnx_results
    detected_boxes = results_object.boxes.xyxy.cpu().numpy()
    detected_class_indices = results_object.boxes.cls.cpu().numpy()
    detected_class_names = results_object.names
    
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for i, detected_box in enumerate(detected_boxes):
        detected_class_index = int(detected_class_indices[i])
        detected_class_name = detected_class_names.get(detected_class_index, 'Unknown')
        detected_box_converted = convert_bbox_format(detected_box)
        
        is_true_positive = False
        for j, actual_bbox in enumerate(actual_bboxes):
            actual_label = actual_labels[j]
            iou = calculate_iou(detected_box_converted, actual_bbox)
            
            if iou >= iou_threshold and detected_class_name == detected_class_names.get(actual_label, 'Unknown'):
                is_true_positive = True
                break
        
        if is_true_positive:
            true_positives += 1
        else:
            false_positives += 1

    false_negatives = len(actual_bboxes) - true_positives

    return true_positives, false_positives, false_negatives, len(detected_boxes), len(actual_bboxes)

coco_json_file_path = 'test_annotations.json'
coco_annotations = load_coco_annotations(coco_json_file_path)

total_true_positives = 0
total_false_positives = 0
total_false_negatives = 0
total_detected_boxes = 0
total_actual_boxes = 0
total_iou_scores = []

for idx, image_info in enumerate(coco_annotations['images']):
    image_id = image_info['id']
    
    if idx < len(results):
        onnx_results = results[idx]  # Get the results for this specific image
        true_positives, false_positives, false_negatives, detected_boxes, actual_boxes = compare_results(onnx_results, coco_annotations, image_id)
        
        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives
        total_detected_boxes += detected_boxes
        total_actual_boxes += actual_boxes

        # Calculate IoU scores for this image
        iou_scores = []
        for i, detected_box in enumerate(onnx_results.boxes.xyxy.cpu().numpy()):
            detected_class_index = int(onnx_results.boxes.cls[i])
            detected_class_name = onnx_results.names.get(detected_class_index, 'Unknown')
            detected_box_converted = convert_bbox_format(detected_box)

            is_true_positive = False
            for j, actual_bbox in enumerate(get_actual_bbox_and_labels(coco_annotations, image_id)[0]):
                actual_label = get_actual_bbox_and_labels(coco_annotations, image_id)[1][j]
                iou = calculate_iou(detected_box_converted, actual_bbox)

                if iou >= 0.5 and detected_class_name == onnx_results.names.get(actual_label, 'Unknown'):
                    is_true_positive = True
                    break

            if is_true_positive:
                iou_scores.append(iou)

        total_iou_scores.extend(iou_scores)

# Calculate accuracy
if total_detected_boxes > 0:
    precision = total_true_positives / (total_true_positives + total_false_positives)
    recall = total_true_positives / (total_true_positives + total_false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)
    accuracy = f1_score * 100
    print(f"Accuracy: {accuracy:.2f} %")
else:
    print("No results available to calculate accuracy.")

# Calculate average IoU score
average_iou = np.mean(total_iou_scores) if total_iou_scores else 0
print(f"Average IoU score: {average_iou:.4f}")

# Calculate precision, recall, and F1 score
precision = total_true_positives / (total_true_positives + total_false_positives)
recall = total_true_positives / (total_true_positives + total_false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)
print(f"Precision: {precision * 100:.4f} %")
print(f"Recall: {recall * 100:.4f} %")
print(f"F1 Score: {f1_score:.4f}")

Accuracy: 55.77 %
Average IoU score: 0.8608
Precision: 54.3500 %
Recall: 57.2606 %
F1 Score: 0.5577


## **Accuracy based on Correctly Detected Images**

In [63]:
import json

def load_coco_annotations(json_file_path):
    with open(json_file_path, 'r') as f:
        coco_data = json.load(f)
    return coco_data

def get_actual_bbox_and_labels(coco_data, image_id):
    annotations = [anno for anno in coco_data['annotations'] if anno['image_id'] == image_id]
    bboxes = [anno['bbox'] for anno in annotations]
    labels = [anno['category_id'] for anno in annotations]
    return bboxes, labels

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area / union_area
    return iou

def convert_bbox_format(bbox):
    """Convert [x1, y1, x2, y2] to [x, y, width, height]."""
    x1, y1, x2, y2 = bbox
    return [x1, y1, x2 - x1, y2 - y1]

def compare_results(onnx_results, coco_annotations, image_id, iou_threshold=0.5):
    actual_bboxes, actual_labels = get_actual_bbox_and_labels(coco_annotations, image_id)
    
    results_object = onnx_results
    detected_boxes = results_object.boxes.xyxy.cpu().numpy()
    detected_class_indices = results_object.boxes.cls.cpu().numpy()
    detected_class_names = results_object.names
    
    is_image_detected = False

    for i, detected_box in enumerate(detected_boxes):
        detected_class_index = int(detected_class_indices[i])
        detected_class_name = detected_class_names.get(detected_class_index, 'Unknown')
        detected_box_converted = convert_bbox_format(detected_box)
        
        for j, actual_bbox in enumerate(actual_bboxes):
            actual_label = actual_labels[j]
            iou = calculate_iou(detected_box_converted, actual_bbox)
            
            if iou >= iou_threshold and detected_class_name == detected_class_names.get(actual_label, 'Unknown'):
                is_image_detected = True
                break
        
        if is_image_detected:
            break

    return is_image_detected

coco_json_file_path = 'test_annotations.json'
coco_annotations = load_coco_annotations(coco_json_file_path)

correctly_detected_images = 0

for idx, image_info in enumerate(coco_annotations['images']):
    image_id = image_info['id']
    
    if idx < len(results):
        onnx_results = results[idx]  # Get the results for this specific image
        if compare_results(onnx_results, coco_annotations, image_id):
            correctly_detected_images += 1

# Calculate accuracy
total_images = len(coco_annotations['images'])
accuracy = (correctly_detected_images / total_images) * 100

print(f"Number of Correctly Detected Images: {correctly_detected_images} out of 1758 Test Images")
print(f"Accuracy: {accuracy:.2f} %")

Number of Correctly Detected Images: 1044 out of 1758 Test Images
Accuracy: 59.39 %
